# IMPORTING THE LIBRARY

In [1]:
import pandas as pd
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn import set_config

In [2]:
df=pd.read_csv('titanic_train.csv')

In [3]:
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [4]:
df.drop(columns=['PassengerId','Name','Ticket','Cabin'],inplace=True)

In [5]:
df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S
887,1,1,female,19.0,0,0,30.0000,S
888,0,3,female,NaN,1,2,23.4500,S
889,1,1,male,26.0,0,0,30.0000,C


In [6]:
x_train,x_test,y_train,y_test=train_test_split(df.drop(columns=['Survived']),df['Survived'],test_size=0.2)

In [7]:
x_train.head(2)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
356,1,female,22.0,0,1,55.0000,S
638,3,female,41.0,0,5,39.6875,S


In [8]:
x_train.isnull().sum()

Pclass        0
Sex           0
Age         145
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

# applying imputation on missing values

In [9]:
si_age=SimpleImputer()
si_embarked=SimpleImputer(strategy='most_frequent')

In [10]:
tnf1=ColumnTransformer([
 ('age_impute',si_age,[2]),
 ('embarked_impute',si_embarked,[6])   
],remainder='passthrough')

# applying one hot encoder in categorical features

In [11]:
ohe=OneHotEncoder(sparse=False,handle_unknown='ignore')

In [12]:
tnf2=ColumnTransformer([
    ('tnf2',ohe,[1,6])
],remainder='passthrough')

# scaling

In [13]:
scale=MinMaxScaler()

In [14]:
tnf3=ColumnTransformer([
    ('scale',scale,slice(1,10))
])

# feature selection

In [15]:
tnf4=SelectKBest(score_func=chi2,k=6)

In [16]:
#train the model
tnf5=DecisionTreeClassifier()

# create pipeline

In [17]:
pipe=Pipeline([
  ('trnsf1',tnf1),
  ('trnsf2',tnf2),
    ('trnsf3',tnf3),
    ('trnsf4',tnf4),
    ('trnsf5',tnf5)
])  


In [24]:
#train
pipe.fit(x_train,y_train)

Pipeline(steps=[('trnsf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('age_impute', SimpleImputer(),
                                                  [2]),
                                                 ('embarked_impute',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trnsf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('tnf2',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 6])])),
                ('trnsf3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(1, 10, None))])),
                ('trnsf4',
                 SelectKBest(k=6,
                             score_func=<function chi2 at 0x000001D4060FCAF0>)),
                ('trnsf5', DecisionTreeClassifier())])

In [19]:
#display pipeline

set_config(display='diagram')

In [20]:
#PREDICT
y_pred=pipe.predict(x_test)

In [21]:
y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0], dtype=int64)

In [22]:
y_test

865    1
744    1
810    0
346    1
423    0
      ..
531    0
822    0
501    0
526    1
845    0
Name: Survived, Length: 179, dtype: int64

In [23]:

accuracy_score(y_test,y_pred)

0.5921787709497207